In [ ]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None)): 
  ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
def batch_selector(x,y,batch_size,mode="random"):
    if(mode=="random"):
        indices=np.random.choice(len(x),batch_size)
        return(x[indices],y[indices],indices)
    

In [ ]:
def plotit(y,labelx,labely,title):
    plt.plot(y)
    plt.xlabel(labelx)
    plt.ylabel(labely)
    plt.suptitle(title)
    plt.show()

In [ ]:
class NN:
    def __init__ (self,I,H,O):
        self.I=I
        self.H=H
        self.O=O
        self.W1 = tf.Variable(tf.random_normal([self.H[-1], self.O]))
        self.b1 = tf.Variable(tf.ones([self.O]))
        self.x_ = tf.placeholder(tf.float32, [None, self.I])
        self.x=[self.x_]
        self.W = [tf.Variable(tf.random_normal([self.I, self.H[0]]))]+[tf.Variable(tf.random_normal([self.H[HH], self.H[HH+1]])) for HH in range(len(self.H)-1)]
        self.b = [tf.Variable(tf.ones([HH])) for HH in self.H]
        for ii in range(len(self.H)):
            self.x=self.x+[tf.nn.sigmoid(tf.matmul(self.x[-1], self.W[ii]) + self.b[ii])]

        self.y_logit=tf.matmul(self.x[-1], self.W1) + self.b1
        self.y = tf.nn.softmax(self.y_logit)
        self.y_ = tf.placeholder(tf.float32, [None, self.O])
        self.cross_entropy1 = (-tf.reduce_sum(self.y_ * tf.log(self.y), reduction_indices=[1]))
        self.cross_entropy=tf.reduce_mean(self.cross_entropy1)
        self.uninitialized_vars=None
        self.train_step = tf.train.GradientDescentOptimizer(0.3).minimize(self.cross_entropy,var_list=self.uninitialized_vars)
        self.correct_prediction = tf.equal(tf.argmax(self.y,1), tf.argmax(self.y_,1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))
        self.WW = []
        self.bb = []
        self.WW1 = []
        self.bb1 = []
        self.xx=[[self.x_]]
    def addh(self):
        self.WW = self.WW+[[tf.Variable(tf.random_normal([self.I, self.H[0]]))]+[tf.Variable(tf.random_normal([self.H[HH], self.H[HH+1]])) for HH in range(len(self.H)-1)]]
        self.bb = self.bb+[[tf.Variable(tf.ones([HH])) for HH in self.H]]
        self.xx=self.xx+[[self.x_]]
        for ii in range(len(self.H)):
            self.xx[-1]=self.xx[-1]+[tf.nn.sigmoid(tf.matmul(self.xx[-1][-1], self.WW[-1][ii]) + self.bb[-1][ii])]
        self.WW1 = self.WW1+[tf.Variable(tf.zeros([self.H[-1], self.O]))]
        self.bb1 = self.bb1+[tf.Variable(tf.ones([self.O]))]
        self.y_logit=self.y_logit+tf.matmul(self.xx[-1][-1], self.WW1[-1]) + self.bb1[-1]
        uninitialized_vars = []
        for var in tf.global_variables():
            try:
                sess.run(var)
            except tf.errors.FailedPreconditionError:
                uninitialized_vars.append(var)

        init_new_vars_op = tf.variables_initializer(uninitialized_vars)
        #sess.run(tf.variables_initializer(sess.run(tf.report_uninitialized_variables())))
        sess.run(init_new_vars_op)
        self.train_step = tf.train.GradientDescentOptimizer(0.3).minimize(self.cross_entropy,var_list=self.uninitialized_vars)
        #tf.assign_add(self.y_logit,(tf.matmul(self.xx[-1][-1], self.WW1[-1]) + self.bb1[-1]))
        #tf.variables_initializer((self.WW1[-1],self.bb1[-1],self.WW[-1],self.bb[-1]))
        
        
        

In [ ]:
ff=NN(784,[16,16],10)

In [ ]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
batch_size=10;accumulator1=[]

In [ ]:
for _ in range(1000):
    batch_xs, batch_ys,indices = batch_selector(mnist.train.images,mnist.train.labels,batch_size,'random')
    sess.run(ff.train_step, feed_dict={ff.x_: batch_xs, ff.y_: batch_ys})
    if _%10==0:
        accumulator1=accumulator1+[sess.run(ff.cross_entropy, feed_dict={ff.x_: mnist.train.images, ff.y_: mnist.train.labels})]

        
print(sess.run(ff.accuracy, feed_dict={ff.x_: mnist.test.images, ff.y_: mnist.test.labels}))

In [ ]:
ff.addh()

In [ ]:
#with tf.Session() as sess:
   # batch_xs, batch_ys,indices = batch_selector(mnist.train.images,mnist.train.labels,batch_size,'random')
  #  tf.global_variables_initializer().run()
  #  writer = tf.summary.FileWriter('logs', sess.graph)
  #  print(sess.run(ff.train_step, feed_dict={ff.x_: batch_xs, ff.y_: batch_ys}))
  #  writer.close()

In [ ]:
for _ in range(1000):
    batch_xs, batch_ys,indices = batch_selector(mnist.train.images,mnist.train.labels,batch_size,'random')
    sess.run(ff.train_step, feed_dict={ff.x_: batch_xs, ff.y_: batch_ys})
    if _%10==0:
        accumulator1=accumulator1+[sess.run(ff.cross_entropy, feed_dict={ff.x_: mnist.train.images, ff.y_: mnist.train.labels})]

In [ ]:
print(sess.run(ff.accuracy, feed_dict={ff.x_: mnist.test.images, ff.y_: mnist.test.labels}))

In [ ]:
ff.addh()

In [ ]:
for _ in range(1000):
    batch_xs, batch_ys,indices = batch_selector(mnist.train.images,mnist.train.labels,batch_size,'random')
    sess.run(ff.train_step, feed_dict={ff.x_: batch_xs, ff.y_: batch_ys})
    if _%10==0:
        accumulator1=accumulator1+[sess.run(ff.cross_entropy, feed_dict={ff.x_: mnist.train.images, ff.y_: mnist.train.labels})]

In [ ]:
print(sess.run(ff.accuracy, feed_dict={ff.x_: mnist.test.images, ff.y_: mnist.test.labels}))

In [ ]:
plotit(accumulator1,"Number of iterations/10","Training loss","Training loss vs Training time")

In [ ]:
sess.run(ff.accuracy, feed_dict={ff.x_: mnist.test.images, ff.y_: mnist.test.labels})

In [ ]:
#concatenation not done yet